In [16]:
import os
import sys
while os.path.split(os.getcwd())[1] != 'RecSysChallenge2023-Team':
    os.chdir('..')
sys.path.insert(1, os.getcwd())

import Daniele.Utils.MatrixManipulation as mm
import Daniele.Utils.MyDataManager as dm

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.preprocessing import normalize
import similaripy

URMv = dm.getURMviews()
URMo = dm.getURMopen()
URM_all = mm.defaultExplicitURM(urmv=URMv, urmo=URMo, normalize=False, add_aug=False)

ICMt=dm.getICMt()
ICMl=dm.getICMl()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, [10])
evaluator_test = EvaluatorHoldout(URM_test, [10])

100%|██████████| 663370/663370 [00:00<00:00, 783364.13it/s]


Deallocating Cython objects
EvaluatorHoldout: Ignoring 1148 ( 1.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 314 ( 0.4%) Users that have less than 1 test interactions


In [17]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

recommender = SLIM_BPR_Cython(URM_train=URM_train)
recommender.fit(epochs=200, symmetric=True, lambda_i=0.552113, lambda_j=0.444161, learning_rate=0.02174)

SLIM_BPR_Recommender: URM Detected 1 ( 0.0%) users with no interactions.
SLIM_BPR_Recommender: URM Detected 3461 (12.4%) items with no interactions.
Deallocating Cython objects
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.


/bin/sh: free: command not found


Processed 74379 (100.0%) in 1.31 sec. BPR loss is 8.12E-03. Sample per second: 56805
SLIM_BPR_Recommender: Epoch 1 of 200. Elapsed time 0.32 sec
Processed 74379 (100.0%) in 0.53 sec. BPR loss is 1.98E-02. Sample per second: 139990
SLIM_BPR_Recommender: Epoch 2 of 200. Elapsed time 0.54 sec
Processed 74379 (100.0%) in 0.72 sec. BPR loss is 2.77E-02. Sample per second: 102863
SLIM_BPR_Recommender: Epoch 3 of 200. Elapsed time 0.74 sec
Processed 74379 (100.0%) in 0.91 sec. BPR loss is 3.40E-02. Sample per second: 81436
SLIM_BPR_Recommender: Epoch 4 of 200. Elapsed time 0.93 sec
Processed 74379 (100.0%) in 1.10 sec. BPR loss is 3.86E-02. Sample per second: 67481
SLIM_BPR_Recommender: Epoch 5 of 200. Elapsed time 1.11 sec
Processed 74379 (100.0%) in 0.29 sec. BPR loss is 4.29E-02. Sample per second: 256990
SLIM_BPR_Recommender: Epoch 6 of 200. Elapsed time 1.30 sec
Processed 74379 (100.0%) in 0.48 sec. BPR loss is 4.67E-02. Sample per second: 155317
SLIM_BPR_Recommender: Epoch 7 of 200. Ela

In [18]:
result, result_string = evaluator_validation.evaluateRecommender(recommender)

EvaluatorHoldout: Processed 73231 (100.0%) in 48.42 sec. Users per second: 1512


In [19]:
result

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.061066,0.134537,0.132273,0.036722,0.078983,0.245227,0.132281,0.083556,0.412271,0.300033,...,0.984566,0.405908,0.984566,0.508784,14.046266,1.000241,1.155389,1.043047,0.169474,0.61192


In [14]:
result_test, result_test_string = evaluator_test.evaluateRecommender(recommender)

EvaluatorHoldout: Processed 73467 (100.0%) in 52.00 sec. Users per second: 1413


In [15]:
result_test

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.070565,0.147511,0.144668,0.046104,0.094054,0.290055,0.153246,0.09486,0.44978,0.365552,...,0.987738,0.444265,0.987738,0.490863,13.994219,1.000236,1.111415,1.039177,0.166488,0.537435
